# 1. Setup

## 1-1. Library Setup

In [24]:
import os
import numpy as np
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.model_selection import StratifiedKFold
from keras.utils import np_utils

In [25]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11137504826349692394,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3665166336
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4306344299609454633
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"]

In [26]:
!nvidia-smi

Tue Dec 14 08:04:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   52C    P8    15W /  N/A |   1559MiB /  6144MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
from google.colab import drive
drive.mount('/content/gdrive/')

"\nfrom google.colab import drive\ndrive.mount('/content/gdrive/')\n"

## 1-2. parameter setup

In [28]:
class_num = 2
batch_size = 64

#base_path = ''
base_path = 'gdrive/My Drive/Research_1_1_1/'

path_data_image = base_path  + 'data_image/'
path_data_numpy = base_path + 'data_numpy/'

In [29]:
path_list=[path_data_image, path_data_numpy]
path_list

['data_image/', 'data_numpy/']

In [7]:
def createDirectory(name): 
    try: 
        if not os.path.exists(name): 
            os.makedirs(name) 
    except OSError: 
        print("Error: Failed to create the directory.")  
        
for i in path_list:
    createDirectory(i)

# 2. Image to numpy

## 2-1. Image to numpy 함수 정의

In [8]:
def loading_image_to_numpy(image_folder_name):
    
    file_list_detail = []
    file_list = os.listdir( path_data_image + image_folder_name )

    print (file_list)
    
    for i in file_list:
        file_list_detail_temp = os.listdir( path_data_image + image_folder_name + '/' + i + "/" )
        for j in file_list_detail_temp:
            file_list_detail.append( path_data_image + image_folder_name + '/' + i + "/" + j ) 
            
    print(file_list_detail[:5])
    print("file_list_detail len : ", len(file_list_detail))
    
    temp_dictionary = {k:ind for ind, k in enumerate(file_list)}
    print("temp_dictionary : ", temp_dictionary)
    
    for i, f in enumerate(file_list_detail):   
        temp1, temp2, label, _ = f.split('/')

        # 이미지 불러오기, 사이즈 리사이즈 128 x 128
        new_img = load_img(f,target_size=(128,128)) 
        # 이미지를 array 형식으로 변형
        arr_img = img_to_array(new_img)
        # 차원을 하나 더 추가한다. 4차원 img = (1, 128, 128, 3)
        img = arr_img.reshape((1,)+arr_img.shape)

        # 첫 번째 데이터의 경우 container를 생성
        if i == 0 :
            container = img
            labels = temp_dictionary[label]

        # 첫 번쨰가 아닌경우 array에 계속 쌓아나감
        else:
            container = np.vstack([container, img])
            labels = np.vstack([labels, temp_dictionary[label]])

        print (i)
        
    return (container, labels)

## 2-2. Source Domain Image

In [9]:
image_folder_name_S = 'image-S'

container_S, labels_S = loading_image_to_numpy(image_folder_name_S)

['S0', 'S1']
['data_image/image-S/S0/V006_77_0_00_02_03_11_0_b02_20201021_0000_S01_1.jpg', 'data_image/image-S/S0/V006_77_0_00_02_03_11_0_b02_20201021_0001_S01_1.jpg', 'data_image/image-S/S0/V006_77_0_00_02_03_11_0_b02_20201021_0002_S01_1.jpg', 'data_image/image-S/S0/V006_77_0_00_02_03_11_0_b02_20201021_0003_S01_1.jpg', 'data_image/image-S/S0/V006_77_0_00_02_03_11_0_b02_20201021_0004_S01_1.jpg']
file_list_detail len :  3200
temp_dictionary :  {'S0': 0, 'S1': 1}
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160


1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967


In [10]:
print(container_S.shape, labels_S.shape)
unique_S, counts_S = np.unique(labels_S, return_counts=True)
print(dict(zip(unique_S, counts_S)))

(3200, 128, 128, 3) (3200, 1)
{0: 1600, 1: 1600}


In [11]:
np.save( path_data_image + 'container_S.npy', container_S)
np.save( path_data_image + 'labels_S.npy', labels_S)

##  2-3. TARGET  Domain Image

In [12]:
image_folder_name_T = 'image-T'

container_T, labels_T = loading_image_to_numpy(image_folder_name_T)

['T0', 'T1']
['data_image/image-T/T0/V006_77_0_00_03_03_11_0_c15_20201119_0000_S01_1.jpg', 'data_image/image-T/T0/V006_77_0_00_03_03_11_0_c15_20201119_0001_S01_1.jpg', 'data_image/image-T/T0/V006_77_0_00_03_03_11_0_c15_20201119_0002_S01_1.jpg', 'data_image/image-T/T0/V006_77_0_00_03_03_11_0_c15_20201119_0003_S01_1.jpg', 'data_image/image-T/T0/V006_77_0_00_03_03_11_0_c15_20201119_0004_S01_1.jpg']
file_list_detail len :  2400
temp_dictionary :  {'T0': 0, 'T1': 1}
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160


1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967


In [13]:
print(container_T.shape, labels_T.shape )
unique_T, counts_T = np.unique(labels_T, return_counts=True)
print(dict(zip(unique_T, counts_T)))

(2400, 128, 128, 3) (2400, 1)
{0: 1200, 1: 1200}


In [14]:
np.save( path_data_image + 'container_T.npy', container_T)
np.save( path_data_image + 'labels_T.npy', labels_T)

# 3. Train set, Valid set, Test set 분리

## 3-1. 함수 정의

In [30]:
index_name_xTrain_S = 'xTrain_S'
index_name_xValid_S = 'xValid_S'
index_name_xTest_S = 'xTest_S'

index_name_xTrain_T = 'xTrain_T'
index_name_xValid_T = 'xValid_T'
index_name_xTest_T = 'xTest_T'

def shuffle(xTrain, yTrain, index_name):
    Index = np.arange(yTrain.shape[0])
    print("Index initial value : ", Index[:10])
    
    np.random.shuffle(Index)
    print("Index shuffle : ", Index[:10])
    
    print("yTrain : ")
    print(yTrain[:10])
    
    temp_zero_index = Index[0]
    temp_xTrain = xTrain[temp_zero_index]
    temp_yTrain = yTrain[temp_zero_index]    
    
    xTrain = xTrain[Index]
    yTrain = yTrain[Index]

    print("yTrain : ")
    print(yTrain[:10])
    
    print("-----------------------------")
    print(temp_xTrain)
    print("-----------------------------")
    print(xTrain[0])
    print("-----------------------------")
    print(temp_yTrain, yTrain[0])
    print("-----------------------------")
    
    
    unique, counts = np.unique(yTrain[:batch_size], return_counts=True)
    print(dict(zip(unique, counts)))

    Index_name_path = path_data_numpy + 'Index_suffle_' + index_name + '.npy'
    np.save( Index_name_path, Index)
    
    return (xTrain, yTrain)

## 3-2. Source Domain Data

In [26]:
container_S = np.load( path_data_image + 'container_S.npy' )
labels_S = np.load( path_data_image + 'labels_S.npy' )

In [27]:
from sklearn.model_selection import train_test_split

#1.1 dataset 분리 train_test_split - train / test (6:4)
xTrain_S, xTest_S, yTrain_S, yTest_S = train_test_split(container_S, labels_S, train_size=0.6, 
                                                     random_state=1, stratify=labels_S)
#1.2 dataset 분리 train_test_split - test / validation  (5:5)
xTest_S, xValid_S, yTest_S, yValid_S = train_test_split(xTest_S, yTest_S, test_size=0.5, 
                                                 random_state=1, stratify=yTest_S)

In [28]:
print(container_S.shape, labels_S.shape)
print(xTrain_S.shape, yTrain_S.shape)
print(xValid_S.shape, yValid_S.shape)
print(xTest_S.shape, yTest_S.shape)

(3200, 128, 128, 3) (3200, 1)
(1920, 128, 128, 3) (1920, 1)
(640, 128, 128, 3) (640, 1)
(640, 128, 128, 3) (640, 1)


In [25]:
labels_S_unique, labels_S_counts = np.unique(labels_S, return_counts=True)
print(dict(zip(labels_S_unique, labels_S_counts)))
yTrain_S_unique, yTrain_S_counts = np.unique(yTrain_S, return_counts=True)
print(dict(zip(yTrain_S_unique, yTrain_S_counts)))
yValid_S_unique, yValid_S_counts = np.unique(yValid_S, return_counts=True)
print(dict(zip(yValid_S_unique, yValid_S_counts)))
yTest_S_unique, yTest_S_counts = np.unique(yTest_S, return_counts=True)
print(dict(zip(yTest_S_unique, yTest_S_counts)))

{0: 1600, 1: 1600}
{0: 320, 1: 320}
{0: 320, 1: 320}
{0: 640, 1: 640}


In [31]:
xTrain_S, yTrain_S = shuffle(xTrain_S, yTrain_S, index_name_xTrain_S)

Index initial value :  [0 1 2 3 4 5 6 7 8 9]
Index shuffle :  [ 529 1442  540 1233 1271   54  140  607  553 1168]
yTrain : 
[[0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]]
yTrain : 
[[0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]]
-----------------------------
[[[158. 181. 137.]
  [132. 162.  98.]
  [140. 174. 124.]
  ...
  [130. 152.  90.]
  [154. 177. 148.]
  [156. 174. 148.]]

 [[143. 166. 122.]
  [151. 174. 128.]
  [155. 183. 134.]
  ...
  [142. 169. 134.]
  [155. 186. 155.]
  [166. 187. 170.]]

 [[ 92. 133.  55.]
  [124. 140. 111.]
  [126. 157.  98.]
  ...
  [121. 153. 116.]
  [143. 175. 138.]
  [120. 152. 113.]]

 ...

 [[229. 214. 173.]
  [227. 214. 172.]
  [225. 215. 179.]
  ...
  [136. 165. 109.]
  [108. 164.  39.]
  [ 91. 133.  57.]]

 [[209. 195. 156.]
  [227. 214. 179.]
  [225. 214. 182.]
  ...
  [198. 214. 178.]
  [145. 178.  97.]
  [ 94. 154.  54.]]

 [[230. 216. 190.]
  [203. 191. 151.]
  [208. 192. 156.]
  ...
  [166. 191. 126.]
  [134. 174. 101.]
  [112. 166.  46.

In [32]:
xValid_S, yValid_S = shuffle(xValid_S, yValid_S, index_name_xValid_S)

Index initial value :  [0 1 2 3 4 5 6 7 8 9]
Index shuffle :  [232  62 463 362  99 606 462 296  45 480]
yTrain : 
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]]
yTrain : 
[[0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]]
-----------------------------
[[[121. 143.  79.]
  [125. 143.  81.]
  [201. 192. 183.]
  ...
  [160. 153. 143.]
  [178. 170. 157.]
  [175. 167. 156.]]

 [[136. 158.  94.]
  [126. 142.  80.]
  [196. 191. 185.]
  ...
  [165. 158. 148.]
  [178. 170. 157.]
  [179. 173. 159.]]

 [[123. 141.  79.]
  [163. 164. 158.]
  [181. 180. 175.]
  ...
  [169. 161. 150.]
  [171. 163. 150.]
  [178. 170. 157.]]

 ...

 [[ 81.  75.  63.]
  [ 71.  63.  50.]
  [ 92.  85.  77.]
  ...
  [175. 155. 131.]
  [178. 158. 134.]
  [180. 160. 136.]]

 [[ 66.  55.  37.]
  [ 69.  66.  49.]
  [ 86.  82.  73.]
  ...
  [178. 156. 132.]
  [190. 170. 145.]
  [180. 160. 136.]]

 [[ 61.  75.  13.]
  [ 53.  69.   4.]
  [ 52.  66.   7.]
  ...
  [173. 151. 127.]
  [159. 138. 111.]
  [179. 157. 133.]]]
------

In [33]:
xTest_S, yTest_S = shuffle(xTest_S, yTest_S, index_name_xTest_S)

Index initial value :  [0 1 2 3 4 5 6 7 8 9]
Index shuffle :  [486 231 106 439 423  91 118 559  32 639]
yTrain : 
[[0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]]
yTrain : 
[[0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]]
-----------------------------
[[[142. 161. 139.]
  [141. 165. 141.]
  [145. 173. 150.]
  ...
  [158. 186. 137.]
  [168. 191. 149.]
  [154. 177. 131.]]

 [[136. 160. 136.]
  [133. 157. 133.]
  [144. 169. 148.]
  ...
  [160. 188. 150.]
  [154. 183. 139.]
  [ 95. 128.  23.]]

 [[152. 177. 147.]
  [146. 173. 142.]
  [138. 166. 143.]
  ...
  [167. 194. 149.]
  [148. 172. 124.]
  [115. 138.  50.]]

 ...

 [[228. 218. 209.]
  [231. 219. 207.]
  [229. 219. 207.]
  ...
  [236. 222. 213.]
  [238. 225. 216.]
  [240. 227. 218.]]

 [[216. 205. 187.]
  [217. 209. 188.]
  [220. 207. 188.]
  ...
  [236. 221. 214.]
  [238. 225. 219.]
  [232. 222. 212.]]

 [[232. 222. 212.]
  [242. 232. 230.]
  [243. 238. 234.]
  ...
  [235. 226. 221.]
  [228. 217. 211.]
  [229. 220. 215.]]]
------

In [34]:
print(xTrain_S.shape, yTrain_S.shape)
print(xValid_S.shape, yValid_S.shape)
print(xTest_S.shape, yTest_S.shape)

(1920, 128, 128, 3) (1920, 1)
(640, 128, 128, 3) (640, 1)
(640, 128, 128, 3) (640, 1)


## 3-2. Target Domain Data

In [43]:
container_T = np.load( path_data_image + 'container_T.npy' )
labels_T = np.load( path_data_image + 'labels_T.npy' )

In [44]:
#1.1 dataset 분리 train_test_split - train / test (6:4)
xTrain_T, xTest_T, yTrain_T, yTest_T = train_test_split(container_T, labels_T, train_size=0.6, 
                                                     random_state=1, stratify=labels_T)
#1.2 dataset 분리 train_test_split - test / validation  (5:5)
xTest_T, xValid_T, yTest_T, yValid_T = train_test_split(xTest_T, yTest_T, test_size=0.5, 
                                                 random_state=1, stratify=yTest_T)

In [45]:
print(container_T.shape, labels_T.shape)
print(xTrain_T.shape, yTrain_T.shape)
print(xValid_T.shape, yValid_T.shape)
print(xTest_T.shape, yTest_T.shape)

(2400, 128, 128, 3) (2400, 1)
(1440, 128, 128, 3) (1440, 1)
(480, 128, 128, 3) (480, 1)
(480, 128, 128, 3) (480, 1)


In [46]:
labels_T_unique, labels_T_counts = np.unique(labels_T, return_counts=True)
print(dict(zip(labels_T_unique, labels_T_counts)))
yTrain_T_unique, yTrain_T_counts = np.unique(yTrain_T, return_counts=True)
print(dict(zip(yTrain_T_unique, yTrain_T_counts)))
yValid_T_unique, yValid_T_counts = np.unique(yValid_T, return_counts=True)
print(dict(zip(yValid_T_unique, yValid_T_counts)))
yTest_T_unique, yTest_T_counts = np.unique(yTest_T, return_counts=True)
print(dict(zip(yTest_T_unique, yTest_T_counts)))

{0: 1200, 1: 1200}
{0: 720, 1: 720}
{0: 240, 1: 240}
{0: 240, 1: 240}


In [47]:
xTrain_T, yTrain_T = shuffle(xTrain_T, yTrain_T, index_name_xTrain_T)

Index initial value :  [0 1 2 3 4 5 6 7 8 9]
Index shuffle :  [ 157  344 1001  272  175 1240 1328  876    8  491]
yTrain : 
[[1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]]
yTrain : 
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]]
-----------------------------
[[[153. 177. 115.]
  [ 64.  59.  55.]
  [ 78.  80.  79.]
  ...
  [197. 184. 165.]
  [201. 188. 171.]
  [199. 187. 165.]]

 [[169. 193. 131.]
  [110. 129.  84.]
  [ 56.  58.  55.]
  ...
  [201. 188. 171.]
  [200. 187. 170.]
  [201. 190. 170.]]

 [[148. 173. 116.]
  [128. 153.  95.]
  [ 28.  30.  25.]
  ...
  [206. 193. 176.]
  [200. 187. 170.]
  [210. 199. 179.]]

 ...

 [[238. 223. 204.]
  [234. 219. 200.]
  [240. 227. 210.]
  ...
  [192. 177. 158.]
  [194. 176. 154.]
  [241. 227. 201.]]

 [[237. 224. 205.]
  [235. 222. 203.]
  [237. 224. 207.]
  ...
  [196. 181. 160.]
  [195. 180. 159.]
  [219. 197. 173.]]

 [[240. 225. 206.]
  [234. 219. 200.]
  [237. 224. 207.]
  ...
  [220. 203. 183.]
  [208. 193. 174.]
  [200. 182. 158.

In [48]:
xValid_T, yValid_T = shuffle(xValid_T, yValid_T, index_name_xValid_T)

Index initial value :  [0 1 2 3 4 5 6 7 8 9]
Index shuffle :  [ 96 143 400 366  81 123 153 258 149 280]
yTrain : 
[[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]]
yTrain : 
[[1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]]
-----------------------------
[[[144. 146. 145.]
  [101. 109. 112.]
  [187. 184. 177.]
  ...
  [253. 254. 248.]
  [139. 133. 135.]
  [248. 241. 233.]]

 [[149. 151. 150.]
  [115. 119. 122.]
  [150. 149. 147.]
  ...
  [243. 232. 228.]
  [224. 219. 216.]
  [253. 255. 241.]]

 [[144. 150. 150.]
  [129. 133. 136.]
  [113. 113. 115.]
  ...
  [210. 202. 199.]
  [252. 246. 232.]
  [190. 185. 182.]]

 ...

 [[ 67. 110.  30.]
  [ 63. 107.  28.]
  [ 56.  99.  20.]
  ...
  [ 66.  72.  86.]
  [ 74.  80.  92.]
  [ 54.  60.  74.]]

 [[ 78. 124.  51.]
  [ 61. 105.  30.]
  [ 71. 114.  35.]
  ...
  [ 74.  84.  93.]
  [ 73.  81.  92.]
  [ 49.  51.  63.]]

 [[ 49.  95.  22.]
  [ 88. 122.  35.]
  [ 98. 133.  39.]
  ...
  [ 58.  66.  79.]
  [ 44.  56.  70.]
  [ 77.  86.  95.]]]
------

In [49]:
xTest_T, yTest_T = shuffle(xTest_T, yTest_T, index_name_xTest_T )

Index initial value :  [0 1 2 3 4 5 6 7 8 9]
Index shuffle :  [ 29 123 330 339  68 110 144 189  75 244]
yTrain : 
[[0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]]
yTrain : 
[[0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]]
-----------------------------
[[[118. 162. 101.]
  [122. 166. 107.]
  [139. 176. 133.]
  ...
  [106. 162.  71.]
  [107. 163.  76.]
  [108. 163.  70.]]

 [[115. 165. 104.]
  [112. 155.  99.]
  [122. 167. 112.]
  ...
  [105. 164.  70.]
  [102. 163.  70.]
  [109. 165.  78.]]

 [[124. 167. 114.]
  [134. 177. 123.]
  [138. 177. 122.]
  ...
  [109. 165.  76.]
  [104. 160.  69.]
  [109. 168.  88.]]

 ...

 [[108. 135.  28.]
  [105. 134.  26.]
  [ 99. 131.  22.]
  ...
  [ 64. 113.  22.]
  [ 81. 131.  36.]
  [ 96. 136.  74.]]

 [[107. 136.  26.]
  [106. 135.  27.]
  [105. 134.  28.]
  ...
  [ 74. 116.  32.]
  [ 77. 126.  44.]
  [108. 143.  85.]]

 [[102. 130.  19.]
  [114. 140.  33.]
  [106. 135.  25.]
  ...
  [107. 143.  73.]
  [101. 136.  68.]
  [104. 138.  77.]]]
------

In [50]:
print(xTrain_T.shape, yTrain_T.shape)
print(xValid_T.shape, yValid_T.shape)
print(xTest_T.shape, yTest_T.shape)

(1440, 128, 128, 3) (1440, 1)
(480, 128, 128, 3) (480, 1)
(480, 128, 128, 3) (480, 1)


## 3-4. Data Save

In [51]:
print(xTrain_S.shape, yTrain_S.shape)
print(xValid_S.shape, yValid_S.shape)
print(xTest_S.shape, yTest_S.shape)
print()
print(xTrain_T.shape, yTrain_T.shape)
print(xValid_T.shape, yValid_T.shape)
print(xTest_T.shape, yTest_T.shape)

(1920, 128, 128, 3) (1920, 1)
(640, 128, 128, 3) (640, 1)
(640, 128, 128, 3) (640, 1)

(1440, 128, 128, 3) (1440, 1)
(480, 128, 128, 3) (480, 1)
(480, 128, 128, 3) (480, 1)


In [52]:
np.save(path_data_numpy + 'xTrain_S.npy', xTrain_S)
np.save(path_data_numpy + 'xValid_S.npy', xValid_S)
np.save(path_data_numpy + 'xTest_S.npy', xTest_S)

np.save(path_data_numpy + 'yTrain_S.npy', yTrain_S)
np.save(path_data_numpy + 'yValid_S.npy', yValid_S)
np.save(path_data_numpy + 'yTest_S.npy', yTest_S)

np.save(path_data_numpy + 'xTrain_T.npy', xTrain_T)
np.save(path_data_numpy + 'xValid_T.npy', xValid_T)
np.save(path_data_numpy + 'xTest_T.npy', xTest_T)

np.save(path_data_numpy + 'yTrain_T.npy', yTrain_T)
np.save(path_data_numpy + 'yValid_T.npy', yValid_T)
np.save(path_data_numpy + 'yTest_T.npy', yTest_T)

# 4. DA Training Data set 정의

## 4-1. Data Load 및 전처리

In [30]:
xTrain_S = np.load(path_data_numpy + 'xTrain_S.npy')
xValid_S = np.load(path_data_numpy + 'xValid_S.npy')
xTest_S = np.load(path_data_numpy + 'xTest_S.npy')

yTrain_S = np.load(path_data_numpy + 'yTrain_S.npy')
yValid_S = np.load(path_data_numpy + 'yValid_S.npy')
yTest_S = np.load(path_data_numpy + 'yTest_S.npy')

xTrain_T = np.load(path_data_numpy + 'xTrain_T.npy')
xValid_T = np.load(path_data_numpy + 'xValid_T.npy')
xTest_T = np.load(path_data_numpy + 'xTest_T.npy')

yTrain_T = np.load(path_data_numpy + 'yTrain_T.npy')
yValid_T = np.load(path_data_numpy + 'yValid_T.npy')
yTest_T = np.load(path_data_numpy + 'yTest_T.npy')

In [54]:
yTrain_T_valid = np.load(path_data_numpy + 'yTrain_T.npy')

unique, counts = np.unique(yTrain_T_valid, return_counts=True)
print(dict(zip(unique, counts)))

print (yTrain_T_valid[:10])
yTrain_T_valid.fill(0)
print ('--------------------------')
print (yTrain_T_valid[:10])

unique, counts = np.unique(yTrain_T_valid, return_counts=True)
print(dict(zip(unique, counts)))

{0: 720, 1: 720}
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]]
--------------------------
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
{0: 1440}


In [55]:
yValid_T_valid = np.load(path_data_numpy + 'yValid_T.npy')

unique, counts = np.unique(yValid_T_valid, return_counts=True)
print(dict(zip(unique, counts)))

print (yValid_T_valid[:10])
yValid_T_valid.fill(0)
print ('--------------------------')
print (yValid_T_valid[:10])

unique, counts = np.unique(yValid_T_valid, return_counts=True)
print(dict(zip(unique, counts)))

{0: 240, 1: 240}
[[1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]]
--------------------------
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
{0: 480}


In [56]:
print(xTrain_S.shape, yTrain_S.shape)
print(xValid_S.shape, yValid_S.shape)
print(xTest_S.shape, yTest_S.shape)
print()
print(xTrain_T.shape, yTrain_T.shape)
print(xValid_T.shape, yValid_T.shape)
print(xTest_T.shape, yTest_T.shape)
print()
print(yTrain_T_valid.shape)
print(yValid_T_valid.shape)

(1920, 128, 128, 3) (1920, 1)
(640, 128, 128, 3) (640, 1)
(640, 128, 128, 3) (640, 1)

(1440, 128, 128, 3) (1440, 1)
(480, 128, 128, 3) (480, 1)
(480, 128, 128, 3) (480, 1)

(1440, 1)
(480, 1)


In [57]:
xTrain_S = xTrain_S.astype('float32')/255
xValid_S = xValid_S.astype('float32')/255
xTest_S = xTest_S.astype('float32')/255

yTrain_S = np_utils.to_categorical(yTrain_S, class_num)
yValid_S = np_utils.to_categorical(yValid_S, class_num)
yTest_S = np_utils.to_categorical(yTest_S, class_num)

xTrain_T = xTrain_T.astype('float32')/255
xValid_T = xValid_T.astype('float32')/255
xTest_T = xTest_T.astype('float32')/255

yTrain_T = np_utils.to_categorical(yTrain_T, class_num)
yValid_T = np_utils.to_categorical(yValid_T, class_num)
yTest_T = np_utils.to_categorical(yTest_T, class_num)

yTrain_T_valid = np_utils.to_categorical(yTrain_T_valid, class_num)
yValid_T_valid = np_utils.to_categorical(yValid_T_valid, class_num)

In [58]:
print(xTrain_S.shape, yTrain_S.shape)
print(xValid_S.shape, yValid_S.shape)
print(xTest_S.shape, yTest_S.shape)
print()
print(xTrain_T.shape, yTrain_T.shape)
print(xValid_T.shape, yValid_T.shape)
print(xTest_T.shape, yTest_T.shape)
print()
print(yTrain_T_valid.shape)
print(yValid_T_valid.shape)

(1920, 128, 128, 3) (1920, 2)
(640, 128, 128, 3) (640, 2)
(640, 128, 128, 3) (640, 2)

(1440, 128, 128, 3) (1440, 2)
(480, 128, 128, 3) (480, 2)
(480, 128, 128, 3) (480, 2)

(1440, 2)
(480, 2)


## 4-2. Source Domain, Target Domain 설정, sample_weights 설정

In [12]:
test_S_domain = np.tile([1., 0.], [len(xTest_S), 1])
test_T_domain = np.tile([0., 1.], [len(xTest_T),1])

print(test_S_domain.shape, test_T_domain.shape)

(640, 2) (480, 2)


In [34]:
Valid_S_domain = np.tile([1., 0.], [len(xValid_S), 1])
Valid_T_domain = np.tile([0., 1.], [len(xValid_T),1])

print(Valid_S_domain.shape, Valid_T_domain.shape)

(640, 2) (480, 2)


In [60]:
combined_train_imgs = np.concatenate([xTrain_S, xTrain_T])
combined_train_labels = np.concatenate([yTrain_S, yTrain_T])
combined_train_domain = np.concatenate([np.tile([1., 0.], [len(yTrain_S), 1]),
                                        np.tile([0., 1.], [len(yTrain_T), 1])])

print(combined_train_imgs.shape,combined_train_labels.shape,combined_train_domain.shape)

(3360, 128, 128, 3) (3360, 2) (3360, 2)


In [61]:
combined_Valid_imgs = np.concatenate([xValid_S, xValid_T])
combined_Valid_labels = np.concatenate([yValid_S, yValid_T])
combined_Valid_domain = np.concatenate([np.tile([1., 0.], [len(yValid_S), 1]),
                                        np.tile([0., 1.], [len(yValid_T), 1])])

print(combined_Valid_imgs.shape,combined_Valid_labels.shape,combined_Valid_domain.shape)

(1120, 128, 128, 3) (1120, 2) (1120, 2)


In [62]:
combined_test_imgs = np.concatenate([xTest_S, xTest_T])
combined_test_labels = np.concatenate([yTest_S, yTest_T])
combined_test_domain = np.concatenate([np.tile([1., 0.], [len(xTest_S), 1]),
                                        np.tile([0., 1.], [len(xTest_T), 1])])

print(combined_test_imgs.shape,combined_test_labels.shape,combined_test_domain.shape)

(1120, 128, 128, 3) (1120, 2) (1120, 2)


In [63]:
sample_weights_class_label = np.array(([1]*len(yTrain_S)+[0]*len(yTrain_T)))
print(sample_weights_class_label.shape)
print(sample_weights_class_label)
sample_weights_domain_label = np.array(([1]*len(yTrain_S)+[1]*len(yTrain_T)))
print(sample_weights_domain_label.shape)
print(sample_weights_domain_label)

(3360,)
[1 1 1 ... 0 0 0]
(3360,)
[1 1 1 ... 1 1 1]


In [64]:
combined_train_Index = np.arange(combined_train_imgs.shape[0])
print(combined_train_Index[:10])
print(combined_train_Index.shape)

np.random.shuffle(combined_train_Index)
print(combined_train_Index[:10])

[0 1 2 3 4 5 6 7 8 9]
(3360,)
[2120 1612  159 2881 2943 1453 1816 2711 1629 2057]


In [65]:
combined_Valid_Index = np.arange(combined_Valid_imgs.shape[0])
print(combined_Valid_Index[:10])
print(combined_Valid_Index.shape)

np.random.shuffle(combined_Valid_Index)
print(combined_Valid_Index[:10])

[0 1 2 3 4 5 6 7 8 9]
(1120,)
[ 735 1069  499  419  362   28  976  808  241  935]


In [66]:
combined_test_Index = np.arange(combined_test_imgs.shape[0])
print(combined_test_Index[:10])
print(combined_test_Index.shape)

np.random.shuffle(combined_test_Index)
print(combined_test_Index[:10])

[0 1 2 3 4 5 6 7 8 9]
(1120,)
[ 366  696 1085  792  833  822  796  446  347  886]


In [67]:
combined_train_imgs = combined_train_imgs[combined_train_Index]
combined_train_labels = combined_train_labels[combined_train_Index]
combined_train_domain = combined_train_domain[combined_train_Index]

sample_weights_class_label = sample_weights_class_label[combined_train_Index]
sample_weights_domain_label = sample_weights_domain_label[combined_train_Index]

combined_Valid_imgs = combined_Valid_imgs[combined_Valid_Index]
combined_Valid_labels = combined_Valid_labels[combined_Valid_Index]
combined_Valid_domain = combined_Valid_domain[combined_Valid_Index]

combined_test_imgs = combined_test_imgs[combined_test_Index]
combined_test_labels = combined_test_labels[combined_test_Index]
combined_test_domain = combined_test_domain[combined_test_Index]

## 4-3. 비지도학습 검증 Data set 구성

In [68]:
combined_train_labels_valid = np.concatenate([yTrain_S, yTrain_T_valid])
combined_train_labels_valid = combined_train_labels_valid[combined_train_Index]

# 5. Data Save

In [69]:
np.save(path_data_numpy + 'test_S_domain.npy', test_S_domain)
np.save(path_data_numpy + 'test_T_domain.npy', test_T_domain)

np.save(path_data_numpy + 'Valid_S_domain.npy', Valid_S_domain)
np.save(path_data_numpy + 'Valid_T_domain.npy', Valid_T_domain)

np.save(path_data_numpy + 'combined_train_imgs.npy', combined_train_imgs)
np.save(path_data_numpy + 'combined_train_labels.npy', combined_train_labels)
np.save(path_data_numpy + 'combined_train_domain.npy', combined_train_domain)

np.save(path_data_numpy + 'combined_Valid_imgs.npy', combined_Valid_imgs)
np.save(path_data_numpy + 'combined_Valid_labels.npy', combined_Valid_labels)
np.save(path_data_numpy + 'combined_Valid_domain.npy', combined_Valid_domain)

np.save(path_data_numpy + 'combined_test_imgs.npy', combined_test_imgs)
np.save(path_data_numpy + 'combined_test_labels.npy', combined_test_labels)
np.save(path_data_numpy + 'combined_test_domain.npy', combined_test_domain)

np.save(path_data_numpy + 'sample_weights_class_label.npy', sample_weights_class_label)
np.save(path_data_numpy + 'sample_weights_domain_label.npy', sample_weights_domain_label)

np.save(path_data_numpy + 'combined_train_labels_valid.npy', combined_train_labels_valid)
np.save(path_data_numpy + 'yValid_T_valid .npy', yValid_T_valid )